In [ ]:
import torch
from torch import nn
from torch.utils.data import Subset, DataLoader, random_split
from sklearn.model_selection import StratifiedShuffleSplit
import torchvision
import torchvision.transforms.v2 as v2
import os
import matplotlib.pyplot as plt
import numpy as np
import pytorch_lightning as pl
from torchmetrics import Accuracy
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch import Trainer
import wandb
import numpy as np
import random

AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [ ]:
  
def transforms(augmentation: str = None):
  if not augmentation:
    transform = v2.Compose(
      [v2.Resize((64, 64)),
       v2.ToImage(), v2.ToDtype(torch.float32, scale=True),
       v2.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

    return transform

  elif augmentation == "hflip":
    augment = v2.RandomHorizontalFlip(p=0.5)
  elif augmentation == 'vflip':
    augment = v2.RandomVerticalFlip(p=0.5)
  elif augmentation == 'rotate':
    augment = v2.RandomRotation(degrees=45)

  transform = v2.Compose(
    [v2.Resize((64, 64)),
     v2.ToImage(), v2.ToDtype(torch.float32, scale=True),
     augment,
     v2.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]
  )
  return transform



In [ ]:
import numpy as np
import random

batch_size = 32
NUM_WORKERS = 2
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(42)

!cp -r "/content/drive/MyDrive/MS/Courses/DL_Assignment2/inaturalist_12K" /content/
DATA_DIR = "/content/inaturalist_12K"

AttributeError: module 'torch' has no attribute '_six'

In [ ]:


transform = transforms(augmentation='rotate')
#DATA_DIR = "/content/drive/MyDrive/MS/Courses/DL_Assignment2/inaturalist_12K"
train_dataset_complete = torchvision.datasets.ImageFolder(root=os.path.join(DATA_DIR, "train"), transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root=os.path.join(DATA_DIR, "val"), transform=transform)

# Getting labels and random splitting/shuffling of each class examples
labels = np.array([entry[1] for entry in train_dataset_complete.samples])
split_fn = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 219)
train_ids, valid_ids = next(split_fn.split(np.zeros(len(labels)), labels))

train_dataset = Subset(train_dataset_complete, train_ids)
valid_dataset = Subset(train_dataset_complete, valid_ids)


# Dataloader
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=NUM_WORKERS, pin_memory = True, worker_init_fn=seed_worker,
    generator=g)

val_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=NUM_WORKERS, pin_memory= True, worker_init_fn=seed_worker,
    generator=g)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=NUM_WORKERS, pin_memory= True, worker_init_fn=seed_worker,
    generator=g)

classes = train_dataset_complete.classes
n_classes = len(classes)

NameError: name 'train_dataloader' is not defined

In [ ]:
def imgplot(data):
  images, labels = next(iter(data))
  plt.figure(figsize=(8,8))
  samples = []
  for i in range(len(images)):
    img = images[i]/2 + 0.5
    if labels[i] in samples:
      continue
    else:
      samples.append(labels[i])
      plt.subplot(2,5,len(samples))
      plt.imshow(img.permute(1,2,0))
      plt.axis('off')
      plt.title(classes[labels[i]])
    if len(samples) == n_classes:
      break
  plt.tight_layout()
  plt.show()


imgplot(train_dataloader)


In [ ]:
class CNN(nn.Module):
    def __init__(self, input, filters, kernel, pool_kernel, pool_stride, batchnorm, activation, dropout, ffn_size, num_classes=10):
        super().__init__()

        self.act = self._activation(activation)
        self.convblock1 = self._convblock(input, filters[0], kernel[0], pool_kernel[0], pool_stride[0], activation, batchnorm, dropout)
        self.convblock2 = self._convblock(filters[0], filters[1], kernel[1], pool_kernel[1], pool_stride[1], activation, batchnorm, dropout)
        self.convblock3 = self._convblock(filters[1], filters[2], kernel[2], pool_kernel[2], pool_stride[2], activation, batchnorm, dropout)
        self.convblock4 = self._convblock(filters[2], filters[3], kernel[3], pool_kernel[3], pool_stride[3], activation, batchnorm, dropout)
        self.convblock5 = self._convblock(filters[3], filters[4], kernel[4], pool_kernel[4], pool_stride[4], activation, batchnorm, dropout)
        if batchnorm:
            self.batch_norm = nn.BatchNorm1d(num_features=ffn_size)
        else:
            self.batch_norm = nn.Identity()

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.LazyLinear(ffn_size)
        self.out = nn.Linear(ffn_size, num_classes)

    def _convblock(self, input, output, kernel, pool_kernel, pool_stride, activation,  batchnorm, dropout=0):

        if batchnorm:
          return torch.nn.Sequential(
              nn.Conv2d(input, output, kernel),
              self.act,
              nn.BatchNorm2d(output),
              nn.MaxPool2d(pool_kernel, pool_stride))
        else:
          return torch.nn.Sequential(
              nn.Conv2d(input, output, kernel),
              self.act,
              nn.MaxPool2d(pool_kernel, pool_stride))

    def _activation(self, act):
        if act == 'relu':
            act = nn.ReLU()
        elif act == 'gelu':
            act = nn.GELU()
        elif act == 'selu':
            act = nn.SELU()
        elif act == 'mish':
            act = nn.Mish()
        elif act == 'swish':
            act = nn.SiLU()
        return act

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = self.batch_norm(x)
        x = self.act(x)
        x = self.dropout(x)
        x = self.out(x)
        return x


In [ ]:
class CNN_light(pl.LightningModule):
    def __init__(self, filters, kernel, pool_kernel, pool_stride, batchnorm, activation, dropout, ffn_size, lr =0.05):
        super().__init__()
        self.save_hyperparameters()
        self.model = CNN(input=3, filters=filters, kernel=kernel, pool_kernel=pool_kernel, pool_stride=pool_stride, batchnorm=batchnorm, activation=activation, dropout=dropout, ffn_size=ffn_size, num_classes=10)
        self.train_accuracy = Accuracy(task='multiclass', num_classes=10)
        self.val_accuracy = Accuracy(task='multiclass', num_classes=10)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        acc = self.train_accuracy(logits, y)
        self.log("train loss", loss, on_step = False, on_epoch = True)
        self.log("train accuracy", acc, on_step = False, on_epoch = True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        acc = self.val_accuracy(logits, y)
        self.log("val loss", loss, on_step = False, on_epoch = True)
        self.log("val accuracy", acc, on_step = False, on_epoch = True)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.hparams.lr, momentum=0.9)
        return optimizer

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7e3e1905e550>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7e3e0c89f490, raw_cell="class CNN_light(pl.LightningModule):
    def __ini.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/user/Documents/Courses/DA6401_A2/lightning_test.ipynb#X22sZmlsZQ%3D%3D>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7e3e1905e550>> (for post_run_cell), with arguments args (<ExecutionResult object at 7e3e0d02e6d0, execution_count=158 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7e3e0c89f490, raw_cell="class CNN_light(pl.LightningModule):
    def __ini.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/user/Documents/Courses/DA6401_A2/lightning_test.ipynb#X22sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
model = CNN_light(filters = [8,16,32,64,128], kernel=[3,3,3,5,5], pool_kernel=[2,2,2,3,3], pool_stride=[1,1,1,2,2], batchnorm=True, activation='mish', dropout=0.4, ffn_size=256, lr =0.0001)
logger= WandbLogger(project= 'dlas2', name = "test2",resume="never", id=None)
trainer = pl.Trainer(max_epochs=5,  accelerator="auto",logger=logger, profiler='simple') #precision="16-mixed",
trainer.fit(model, train_dataloader, val_dataloader)

In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'lr': {
            'min': 1e-4,
            'max': 1e-2
        },
        'batch_size': {
            'values': [32, 64]
        },
        'filters': {
            'values': [[8,8,8,8,8], [16,16,16,16,16], [32,32,32,32,32], [64, 64, 64, 64, 64], [16, 32, 64, 128, 256], [256, 128, 64, 32, 16], [16, 32, 64, 32, 16]]
        },
        'kernel': {
            'values': [[3,3,3,3,3], [5,5,5,5,5], [7,7,7,7,7], [3,3,5,5,5]]
        },
        'pool_kernel': {
            'values': [[2,2,2,2,2], [3,3,3,3,3], [2,2,2,3,3]]
        },
        'pool_stride': {
            'values': [[1,1,1,1,1], [1,1,1,2,2],[2,2,2,2,2]]
        },
        'batchnorm': {
            'values': [True, False]
        },
        'activation': {
            'values': ['relu', 'gelu', 'selu', 'mish', 'swish']
        },
        'augmentation': {
            'values': ['hflip', 'vflip', 'rotate', None]
        },
        'dropout': {
            'min': 0.0,
            'max': 0.5
        },
        'ffn_size': {
            'values': [128, 256, 512]
        },

    }
}


In [ ]:
def trainCNN(config=None):
  with wandb.init(config=config):
    config = wandb.config

    # Dataloading
    transform = transforms(augmentation=config.augmentation)
    DATA_DIR = "/content/inaturalist_12K"
    train_dataset_complete = torchvision.datasets.ImageFolder(root=os.path.join(DATA_DIR, "train"), transform=transform)
    test_dataset = torchvision.datasets.ImageFolder(root=os.path.join(DATA_DIR, "val"), transform=transform)

    # Getting labels and random splitting/shuffling of each class examples
    labels = np.array([entry[1] for entry in train_dataset_complete.samples])
    split_fn = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 219)
    train_ids, valid_ids = next(split_fn.split(np.zeros(len(labels)), labels))

    train_dataset = Subset(train_dataset_complete, train_ids)
    valid_dataset = Subset(train_dataset_complete, valid_ids)


    # Dataloader
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=config.batch_size,
                                              shuffle=True, num_workers=NUM_WORKERS, pin_memory = True, worker_init_fn=seed_worker,
        generator=g)

    val_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=config.batch_size,
                                            shuffle=False, num_workers=NUM_WORKERS, pin_memory= True, worker_init_fn=seed_worker,
        generator=g)

    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size,
                                            shuffle=False, num_workers=NUM_WORKERS, pin_memory= True, worker_init_fn=seed_worker,
        generator=g)

    classes = train_dataset_complete.classes
    n_classes = len(classes)


    # Model
    model = CNN_light(filters= config.filters, kernel = config.kernel, pool_kernel=config.pool_kernel, pool_stride=config.pool_stride, batchnorm=config.batchnorm, activation=config.activation, dropout=config.dropout, ffn_size=config.ffn_size, lr=config.lr)
    logger= WandbLogger()
    trainer = pl.Trainer(max_epochs=5,  accelerator="auto",logger=logger, profiler='simple', precision="16-mixed",)
    trainer.fit(model, train_dataloader, val_dataloader)
    wandb.finish()

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="dla2-sweeps")
wandb.agent(sweep_id, function=trainCNN, count=10)